In [10]:
import nbimporter; nbimporter.options["only_defs"] = False
from P0 import compileString
from ST import printSymTab

def runwasm(wasmfile):
    from IPython.core.display import display, Javascript
    display(Javascript("""
    const params = { 
        P0lib: { 
            write: i => this.append_stream({text: '' + i, name: 'stdout'}),
            writeln: () => this.append_stream({text: '\\n', name: 'stdout'}),
            read: () => window.prompt()
        }
    }

    fetch('""" + wasmfile + """') // asynchronously fetch file, return Response object
      .then(response => response.arrayBuffer()) // read the response to completion and stores it in an ArrayBuffer
      .then(code => WebAssembly.compile(code)) // compile (sharable) code.wasm
      .then(module => WebAssembly.instantiate(module, params)) // create an instance with memory
    // .then(instance => instance.exports.program()); // run the main program; not needed if start function specified
    """))
    
def runpywasm(wasmfile):
    import pywasm
    def write(s, i): print(i)
    def writeln(s): print('\n')
    def read(s): return int(input())
    vm = pywasm.load(wasmfile, {'P0lib': {'write': write, 'writeln': writeln, 'read': read}})
  
 
from wasmer import engine, Store, Module, Instance, ImportObject, Function
from wasmer_compiler_cranelift import Compiler

def runwasmer(wasmfile):
    def write(i: int): print(i)
    def writeln(): print('\n')
    def read() -> int: return int(input()) 
    store = Store(engine.JIT(Compiler))
    module = Module(store, open(wasmfile, 'rb').read())
    import_object = ImportObject()
    import_object.register("P0lib", {"write": Function(store, write),
                                     "writeln": Function(store, writeln),"read": Function(store, read)})
    instance = Instance(module, import_object)

### Scanner Tests for Exception

In [12]:
import SC
def scanString(src):
    SC.init(src); syms = []
    while SC.sym != SC.EOF:
        syms.append(('INDENT' if SC.sym == SC.INDENT else \
                     'DEDENT' if SC.sym == SC.DEDENT else \
                     'IDENT' if SC.sym == SC.IDENT else SC.sym, SC.newline))
        SC.getSym()
    return syms

#### Parsing Explicit Exceptions

In [13]:
scanString("""

program p

  if a then
    writeln()
  else
    writeln()
  if a then writeln() else writeln()
""")

[(48, True),
 ('IDENT', False),
 ('INDENT', False),
 (38, True),
 ('IDENT', False),
 (39, False),
 ('INDENT', False),
 ('IDENT', True),
 (23, False),
 (24, False),
 ('DEDENT', False),
 (40, True),
 ('INDENT', True),
 ('IDENT', True),
 (23, False),
 (24, False),
 ('DEDENT', False),
 (38, True),
 ('IDENT', False),
 (39, False),
 ('IDENT', False),
 (23, False),
 (24, False),
 (40, False),
 ('IDENT', False),
 (23, False),
 (24, False),
 ('DEDENT', False)]

In [14]:
scanString("""

program p

  if a then
    throw
  else
    throw
  if a then writeln() else writeln()
""")

[(48, True),
 ('IDENT', False),
 ('INDENT', False),
 (38, True),
 ('IDENT', False),
 (39, False),
 ('INDENT', False),
 (52, True),
 ('DEDENT', True),
 (40, True),
 ('INDENT', True),
 (52, True),
 ('DEDENT', True),
 (38, True),
 ('IDENT', False),
 (39, False),
 ('IDENT', False),
 (23, False),
 (24, False),
 (40, False),
 ('IDENT', False),
 (23, False),
 (24, False),
 ('DEDENT', False)]

In [15]:
scanString("""

program p

  if a then
    throw exception('Just a test')
  else
    throw
  if a then throw exception('Just a test') else writeln()
""")

Exception: line 6 pos 20 illegal character

### Compiler Tests for Exception

In [16]:
# "Throw" can be compiled，but the WebAssembly Code is incorrect
compileString("""
procedure sqrt(x: integer) → (r: integer)
    if x < 0 then throw else r := x
program equationsolver
    var a: integer
        a ← sqrt(2)
        write(a)
""")

(module
(import "P0lib" "write" (func $write (param i32)))
(import "P0lib" "writeln" (func $writeln))
(import "P0lib" "read" (func $read (result i32)))
(func $sqrt (param $x i32) (result i32)
(local $r i32)
(local $0 i32)
local.get $x
i32.const 0
i32.lt_s
if
throw 0
else
local.get $x
local.set $r
end
local.get $r)
(func $program
(local $a i32)
(local $0 i32)
i32.const 2
call $sqrt
local.set $a
local.get $a
call $write
)
(memory 1)
(start $program)
)


In [17]:
!wat2wasm --enable-exceptions test.wat

#### Zero Division Exception

In [18]:
compileString("""

program DividedByZero
    var a, b: integer
        a := (34 × 5 - 170)
        b := (99 div a)
        write(a)
""", 'wasmtest.wat')

In [19]:
!wat2wasm --enable-exceptions wasmtest.wat

In [20]:
runwasm("wasmtest.wat")

<IPython.core.display.Javascript object>

In [21]:
compileString("""

program DividedByZero
    var b: integer
        b := (99 div 0)
        write(b)
""", 'wasmtest.wat')

Exception: integer division or modulo by zero

In [22]:
!wat2wasm --enable-exceptions wasmtest.wat

In [23]:
runwasm("wasmtest.wat")

<IPython.core.display.Javascript object>

#### Access Before Initialization Exception

In [24]:
compileString("""

program AccessBeforeInitialization
    a := 5
    var a: integer
    
""", 'wasmtest.wat')

Exception: line 4 pos 5 undefined identifier a

In [ ]:
!wat2wasm --enable-exceptions wasmtest.wat

In [ ]:
runwasm("wasmtest.wat")

#### Wrong Type Exception

In [26]:
compileString("""

program WrongType
    var a: boolean
        a := 5
""", 'wasmtest.wat')

Exception: line 5 pos 14 incompatible assignment

In [ ]:
!wat2wasm --enable-exceptions wasmtest.wat

In [ ]:
runwasm("wasmtest.wat")

#### Variable Name Not Defined Exception

In [27]:
compileString("""

program bIsNotDefined
    var a: boolean
    b := 5
    write(b)
""", 'wasmtest.wat')

Exception: line 5 pos 5 undefined identifier b

In [ ]:
!wat2wasm --enable-exceptions wasmtest.wat

In [ ]:
runwasm("wasmtest.wat")

#### Index Out of Bounds Exception

In [28]:
compileString("""

var x: [2 .. 4] → integer
program IndexOutOfBounds
  x[7] := 5
  
""", 'wasmtest.wat')

Exception: line 5 pos 6 index out of bounds

In [ ]:
!wat2wasm --enable-exceptions wasmtest.wat

In [ ]:
runwasm("wasmtest.wat")

In [ ]:
compileString("""

var x: [2 .. 4] → integer
program IndexOutOfBounds
    var i: integer
    x[1] := 1 
    x[2] := 1
    x[6] := 1
  
""", 'wasmtest.wat')

In [ ]:
!wat2wasm --enable-exceptions wasmtest.wat

In [ ]:
runwasm("wasmtest.wat")

In [ ]:
compileString("""

var x: [2 .. 4] → integer
program IndexOutOfBounds
    var i: integer
    i := 7
    write(x[i])
  
""", 'wasmtest.wat')

In [ ]:
!wat2wasm --enable-exceptions wasmtest.wat

In [ ]:
runwasm("wasmtest.wat")

#### Syntax Error Exception

In [ ]:
compileString("""

program missingProcedure
  var b: boolean
    b ← integer()
    
""", 'wasmtest.wat')

In [ ]:
!wat2wasm --enable-exceptions wasmtest.wat

In [ ]:
runwasm("wasmtest.wat")

In [ ]:
compileString("""

program incompatibleCall
  var b: boolean
    b ← read()
    
""", 'wasmtest.wat')

In [ ]:
!wat2wasm --enable-exceptions wasmtest.wat

In [ ]:
runwasm("wasmtest.wat")

In [ ]:
compileString("""
procedure sqrt(x: integer) → (r: integer)
    if x < 0 then throw else
        r := 0;
        while (r + 1) × (r + 1) ≤ y do
            r := r + 1
        x := r
procedure quadraticsolution(a, b, c: integer) → (x, y: integer)
    var d: integer
        d ← sqrt(a × a - 4 × a × c)
        x, y := (- b + d) div (2 × a), (- b - d) div (2 × a)
program equationsolver
    var a, b, c, x, y: integer
    var done: boolean
        done := false
        while ¬done do
            try
                a ← read(); b ← read(); c ← read()
                x, y ← quadraticsolution(a, b, c)
                write(x); write(y)
            catch
                done := true
""")
